# Learning-to-rank : pyTerrier - OpenNIR

Dans cette partie, on s'intéresse à l'utilisation de modèles neuronaux pour la recherche d'information. 
Les modèles neuronaux utilisés ont été rassemblés dans la librairie [OpenNIR](https://opennir.net/). 
On explorera également le modèle T5 avec le plugin [monoT5](https://github.com/terrierteam/pyterrier_t5).

In [1]:
!pip install --upgrade python-terrier
!pip install --upgrade git+https://github.com/Georgetown-IR-Lab/OpenNIR
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_t5

     |████████████████████████████████| 92kB 3.6MB/s 
     |████████████████████████████████| 1.1MB 26.8MB/s 
     |████████████████████████████████| 71kB 7.6MB/s 
     |████████████████████████████████| 215kB 26.0MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
     |████████████████████████████████| 6.3MB 37.9MB/s 
     |████████████████████████████████| 645kB 36.5MB/s 
     |████████████████████████████████| 81kB 10.1MB/s 
     |████████████████████████████████| 133kB 42.0MB/s 
     |████████████████████████████████| 1.8MB 38.7MB/s 
     |████████████████████████████████| 296kB 35.6MB/s 
  Created wheel for python-terrier: filename=python_terrier-0.6.0-cp37-none-any.whl size=93214 sha256=a00854ac00e2b8abf6b36253e5c6c938e21641a481b938418b6b58a7373687b4
  Stored in directory: /root/.cache/pip/wheels/9f/8d/91/58da79fd0a2850843b7e8c6097a52f9ff7aa85953cc9c8d27f
  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9675 sha256=692dff7914d55471220e88385ccc447246856f

## Initialisation
De façon similaire au TP1, on initialise PyTerrier. Nous allons travailler sur le dataset CORD19. le bloc suivant est une répétition du code en TP1.

In [2]:
import pyterrier as pt
if not pt.started():
    pt.init(tqdm='notebook')
import onir_pt

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


terrier-assemblies 5.5  jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.5  jar not found, downloading to /root/.pyterrier...
Done
PyTerrier 0.6.0 has loaded Terrier 5.5 (built by craigmacdonald on 2021-05-20 13:12)
Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


/usr/local/lib/python3.7/dist-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
import os

dataset = pt.datasets.get_dataset('irds:cord19/trec-covid')
topics = dataset.get_topics(variant='description')
qrels = dataset.get_qrels()

cord19 = pt.datasets.get_dataset('irds:cord19/trec-covid')
pt_index_path = './terrier_cord19'

if not os.path.exists(pt_index_path + "/data.properties"):
    # création de l'index
    indexer = pt.index.IterDictIndexer(pt_index_path)
    index_ref = indexer.index(cord19.get_corpus_iter(), 
                              fields=('abstract',), 
                              meta=('docno',))

else:
    # dans le cas où l'index est déjà créé
    index_ref = pt.IndexRef.of(pt_index_path + "/data.properties")

index = pt.IndexFactory.of(index_ref)

[INFO] If you have a local copy of https://ir.nist.gov/covidSubmit/data/topics-rnd5.xml, you can symlink it here to avoid downloading it again: /root/.ir_datasets/downloads/0307a37b6b9f1a5f233340a769d538ea
[INFO] [starting] https://ir.nist.gov/covidSubmit/data/topics-rnd5.xml
[INFO] [finished] https://ir.nist.gov/covidSubmit/data/topics-rnd5.xml: s] [18.7kB] [330kB/s]
[INFO] If you have a local copy of https://ir.nist.gov/covidSubmit/data/qrels-covid_d5_j0.5-5.txt, you can symlink it here to avoid downloading it again: /root/.ir_datasets/downloads/8138424a59daea0aba751c8a891e5f54
[INFO] [starting] https://ir.nist.gov/covidSubmit/data/qrels-covid_d5_j0.5-5.txt
[INFO] [finished] https://ir.nist.gov/covidSubmit/data/qrels-covid_d5_j0.5-5.txt: [7.25s] [1.14MB] [157kB/s]
[INFO] If you have a local copy of https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv, you can symlink it here to avoid downloading it again: /root/.ir_datasets/downloads/80d664e496b8b7e5

12:51:59.231 [ForkJoinPool-1-worker-3] WARN  o.t.structures.indexing.Indexer - Adding an empty document to the index (6iu1dtyl) - further warnings are suppressed

12:53:18.905 [ForkJoinPool-1-worker-3] WARN  o.t.structures.indexing.Indexer - Indexed 54937 empty documents


In [4]:
print(os.path.exists("terrier_index.zip"))

False


In [5]:
#Si le chargement de la collection est trop long à cause du débit ou autres raisons,
# il est possible de récupérer directement l'index fourni par Terrier
import os

if not os.path.exists("terrier_index.zip"):
  !wget http://www.dcs.gla.ac.uk/~craigm/ecir2021-tutorial/terrier_index.zip
  !unzip -j terrier_index.zip -d terrier_index

index_ref = pt.IndexRef.of("./terrier_index/data.properties")
index = pt.IndexFactory.of(index_ref)

--2021-06-17 12:53:26--  http://www.dcs.gla.ac.uk/~craigm/ecir2021-tutorial/terrier_index.zip
Resolving www.dcs.gla.ac.uk (www.dcs.gla.ac.uk)... 130.209.240.1
Connecting to www.dcs.gla.ac.uk (www.dcs.gla.ac.uk)|130.209.240.1|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42017186 (40M) [application/zip]
Saving to: ‘terrier_index.zip’

terrier_index.zip   100%[===================>]  40.07M  7.66MB/s    in 6.0s    

2021-06-17 12:53:34 (6.69 MB/s) - ‘terrier_index.zip’ saved [42017186/42017186]

Archive:  terrier_index.zip
  inflating: terrier_index/data.lexicon.fsomapfile  
  inflating: terrier_index/data.properties  
  inflating: terrier_index/data.lexicon.fsomaphash  
  inflating: terrier_index/data.lexicon.fsomapid  
  inflating: terrier_index/data.meta.idx  
  inflating: terrier_index/data.direct.bf  
  inflating: terrier_index/data.inverted.bf  
  inflating: terrier_index/data.meta.zdata  
  inflating: terrier_index/data.document.fsarrayfile  


## Modèles de ré-ordonancement neuronaux "from scratch"

Les modèles de ré-ordonnancement dans OpenNIR sont constitués de deux éléments :
*  ranker: un modèle d'ordonnancement (e.g., drmm, knrm, pacrr, ...). Cf la liste des [Rankers](https://opennir.net/rankers.html).
*  vocab : défnit comment le texte est encodé par le modèle (e.g., wordvec_hash, bert, ...). Cela permet ainsi de tester plusieurs méthodes de représentation. Plus de détails concernant le [vocab](https://opennir.net/vocab.html).

Les modèles de réordonnancement s'appuient sur une première étape d'ordonnancement (souvent BM25), récupèrent ensuite les textes des top documents et appliquent ensuite le modèle neuronal.



In [6]:
knrm = onir_pt.reranker('knrm', 'wordvec_hash', text_field='abstract')


config file not found: config
[2021-06-17 12:58:38,877][WordvecHashVocab][DEBUG] [starting] downloading https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip


[2021-06-17 12:59:35,815][onir.util.download][WARNING] no hash provided for https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip; consider adding expected_md5="3cc8839ac3fa9a6187149b1e73328b2a" to ensure data integrity.
[2021-06-17 12:59:35,825][onir.util.download][DEBUG] downloaded https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip [56.10s] [682M] [12.7MB/s]
[2021-06-17 12:59:35,844][WordvecHashVocab][DEBUG] [finished] downloading https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip [56.97s]
[2021-06-17 12:59:35,845][WordvecHashVocab][DEBUG] [starting] extracting vecs
[2021-06-17 13:00:00,593][WordvecHashVocab][DEBUG] [finished] extracting vecs [24.75s]
[2021-06-17 13:00:00,594][WordvecHashVocab][DEBUG] [starting] loading vecs into memory
[2021-06-17 13:02:23,531][WordvecHashVocab][DEBUG] [finished] loading vecs into memory [02:23]
[2021-06-17 13:02:23,683][WordvecHashVocab][DEBUG] [starting] writ

Une fois le modèle chargé, il est nécessaire de mettre en place la pipeline d'évaluation vue dans le tp1. 
Le modèle neuronal n'est pas efficace car il n'a pas été entraîné et utilise des poids aléatoires.

In [7]:
br = pt.BatchRetrieve(index) % 100
pipeline = br >> pt.text.get_text(dataset, 'abstract') >> knrm
pt.Experiment(
    [br, pipeline],
    topics,
    qrels,
    names=['DPH', 'DPH >> KNRM'],
    eval_metrics=["map", "ndcg", 'ndcg_cut.10', 'P.10', 'mrt']
)

[INFO] NumExpr defaulting to 2 threads.


[2021-06-17 13:03:09,514][onir_pt][DEBUG] using GPU (deterministic)
[2021-06-17 13:03:18,506][onir_pt][DEBUG] [starting] batches


[2021-06-17 13:03:22,197][onir_pt][DEBUG] [finished] batches: [3.69s] [1250it] [338.99it/s]


,name,map,ndcg,ndcg_cut.10,P.10,mrt
0,DPH,0.068006,0.165607,0.609058,0.658,102.239852
1,DPH >> KNRM,0.054740,0.145452,0.360856,0.452,347.849081


## Entraînement des modèles sur les jeux de données

Pour entraîner les modèles, il est nécessaire de construire la pipeline de modèles/transformations à utliser et ensuite d'appliquer la fonction .fit() à la pipeline. 
Le code ci-dessous prend beaucoup de temps. Il est donné à titre indicatif si vous souhaitez l'utiliser sur des serveurs adaptés. L'étape suivante permet de charger directement les poids du modèle pré-entraîné à l'avance et mis à disposition de la communauté.

Dans ce qui suit, on utilise le jeu de données MS MARCO medical pour pré-entraîner le modèle qui sera ensuite appliqué sur CORD19. 
Il est également possible de garder seulement le jeu de données CORD19, de le découper en train/val/test et regarder les performances. 

In [ ]:
# Apprentissage du modèle sur des données médicales (MS MARCO medical)
from sklearn.model_selection import train_test_split
train_ds = pt.datasets.get_dataset('irds:msmarco-passage/train/medical')
train_topics, valid_topics = train_test_split(train_ds.get_topics(), test_size=50, random_state=42)

# Indexation de MS MARCO pour la première étape d'ordonnancement et récupérer les textes (pour le ranker openNIR)
indexer = pt.index.IterDictIndexer('./terrier_msmarco-passage')
tr_index_ref = indexer.index(train_ds.get_corpus_iter(), fields=('text',), meta=('docno',))

pipeline = (pt.BatchRetrieve(tr_index_ref) % 100 # récupère les 100 premiers documents
            >> pt.text.get_text(train_ds, 'text') # récupère le texte de ces documents
            >> pt.apply.generic(lambda df: df.rename(columns={'text': 'abstract'})) # renomme la colonne
            >> knrm) # applique le re-ranker


pipeline.fit(
    train_topics,
    train_ds.get_qrels(),
    valid_topics,
    train_ds.get_qrels())

Pour éviter l'étape d'entraînement ici, on utlise une version pré-entraînée.

**Important** : penser à supprimer la mémoire des re-rankers qui vont être mis à jour avec les modèles pré-entrainés.

In [8]:
del knrm # free up the memory before loading a new version of the ranker
knrm = onir_pt.reranker.from_checkpoint('https://macavaney.us/knrm.medmarco.tar.gz', text_field='abstract', expected_md5="d70b1d4f899690dae51161537e69ed5a")

[2021-06-17 13:03:22,696][onir.util.download][DEBUG] downloaded https://macavaney.us/knrm.medmarco.tar.gz s] [1.43k] [1.30MB/s] [md5 hash verified]
[2021-06-17 13:03:22,719][WordvecHashVocab][DEBUG] [starting] reading cached at /root/data/onir/vocab/wordvec_hash/fasttext-wiki-news-300d-1M.p
[2021-06-17 13:04:30,710][WordvecHashVocab][DEBUG] [finished] reading cached at /root/data/onir/vocab/wordvec_hash/fasttext-wiki-news-300d-1M.p [01:08]


Il est ensuire possible de lancer la pipeline d'évaluation avec ce nouveau modèle. Les résultats sont meilleurs !

In [9]:
pipeline = br >> pt.text.get_text(dataset, 'abstract') >> knrm
pt.Experiment(
    [br, pipeline],
    topics,
    qrels,
    names=['DPH', 'DPH >> KNRM'],
    baseline=0,       ## spécifie quelle est le modèle de référence pour calculer les améliorations.
    eval_metrics=["map", "ndcg", 'ndcg_cut.10', 'P.10', 'mrt']
)

[2021-06-17 13:04:42,066][onir_pt][DEBUG] using GPU (deterministic)
[2021-06-17 13:04:42,314][onir_pt][DEBUG] [starting] batches


[2021-06-17 13:04:45,889][onir_pt][DEBUG] [finished] batches: [3.57s] [1250it] [350.03it/s]


,name,map,P.10,ndcg,ndcg_cut.10,mrt,map +,map -,map p-value,P.10 +,P.10 -,P.10 p-value,ndcg +,ndcg -,ndcg p-value,ndcg_cut.10 +,ndcg_cut.10 -,ndcg_cut.10 p-value
0,DPH,0.068006,0.658,0.165607,0.609058,77.418922,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DPH >> KNRM,0.065055,0.600,0.160497,0.533312,164.292696,20.0,30.0,0.096372,12.0,26.0,0.028682,20.0,30.0,0.027746,20.0,30.0,0.006401


**Exercice 1**

On souhaite savoir quel est le nombre de documents pertinents qu'il est nécessaire de retourner à l'étape de pré-ordonnancement pour utiliser ensuite le modèle KNRM. Ecrire la pipeline de traitement permettant de trouver la valeur optimale de ce paramètre (valeurs de 10 à 100 par pas de 10).

Remarque : Ne pas faire de train/test exceptionnellement, on souhaite juste constater la valeur qui maximise sur le jeu de données.


In [10]:
cutoffs = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
dph = pt.BatchRetrieve(index)
res = pt.Experiment(
    [dph % cutoff >> pt.text.get_text(dataset, 'abstract') >> knrm for cutoff in cutoffs],
    dataset.get_topics('description'),
    dataset.get_qrels(),
    names=[f'c={cutoff}' for cutoff in cutoffs],
    eval_metrics=["map", "recip_rank", "ndcg", "ndcg_cut.10", "mrt"]
)
res

[2021-06-17 13:10:49,566][onir_pt][DEBUG] using GPU (deterministic)
[2021-06-17 13:10:49,568][onir_pt][DEBUG] [starting] batches


[2021-06-17 13:10:49,994][onir_pt][DEBUG] [finished] batches: s] [125it] [294.18it/s]
[2021-06-17 13:10:53,013][onir_pt][DEBUG] using GPU (deterministic)
[2021-06-17 13:10:53,015][onir_pt][DEBUG] [starting] batches


[2021-06-17 13:10:53,761][onir_pt][DEBUG] [finished] batches: s] [250it] [335.38it/s]
[2021-06-17 13:10:56,806][onir_pt][DEBUG] using GPU (deterministic)
[2021-06-17 13:10:56,808][onir_pt][DEBUG] [starting] batches


[2021-06-17 13:10:57,914][onir_pt][DEBUG] [finished] batches: [1.11s] [375it] [339.20it/s]
[2021-06-17 13:11:00,767][onir_pt][DEBUG] using GPU (deterministic)
[2021-06-17 13:11:00,769][onir_pt][DEBUG] [starting] batches


[2021-06-17 13:11:02,220][onir_pt][DEBUG] [finished] batches: [1.45s] [500it] [344.77it/s]
[2021-06-17 13:11:05,265][onir_pt][DEBUG] using GPU (deterministic)
[2021-06-17 13:11:05,266][onir_pt][DEBUG] [starting] batches


...

[2021-06-17 13:11:17,794][onir_pt][DEBUG] [finished] batches: [2.51s] [875it] [348.46it/s]
[2021-06-17 13:11:20,786][onir_pt][DEBUG] using GPU (deterministic)
[2021-06-17 13:11:20,788][onir_pt][DEBUG] [starting] batches


[2021-06-17 13:11:23,650][onir_pt][DEBUG] [finished] batches: [2.86s] [1000it] [349.48it/s]
[2021-06-17 13:11:26,612][onir_pt][DEBUG] using GPU (deterministic)
[2021-06-17 13:11:26,614][onir_pt][DEBUG] [starting] batches


[2021-06-17 13:11:29,792][onir_pt][DEBUG] [finished] batches: [3.18s] [1125it] [354.01it/s]
[2021-06-17 13:11:32,797][onir_pt][DEBUG] using GPU (deterministic)
[2021-06-17 13:11:32,799][onir_pt][DEBUG] [starting] batches


[2021-06-17 13:11:36,324][onir_pt][DEBUG] [finished] batches: [3.52s] [1250it] [354.65it/s]


,name,map,recip_rank,ndcg,ndcg_cut.10,mrt
0,c=10,0.011818,0.785333,0.049207,0.596295,64.393072
1,c=20,0.020960,0.797500,0.071991,0.585919,72.503793
2,c=30,0.029337,0.805667,0.089965,0.598224,80.423479
3,c=40,0.034766,0.803524,0.102680,0.578190,83.436773
4,c=50,0.040644,0.820667,0.114596,0.590963,94.103628
5,c=60,0.046914,0.810667,0.125965,0.587072,102.124375
6,c=70,0.051899,0.816333,0.136310,0.578087,106.917538
7,c=80,0.056869,0.804667,0.144962,0.565164,114.414075
8,c=90,0.061794,0.806333,0.154542,0.570981,120.085217
9,c=100,0.065055,0.767889,0.160497,0.533312,127.837828


## Modèle Vanilla BERT

**Exercice 2**

Sur le même principe que le modèle KNRM, analyser les performances du modèle vanilla BERT sans et avec pré-entraînement.
Pour la version du modèle pré-entraîné, on utilisera le checkpoint ['https://macavaney.us/scibert-medmarco.tar.gz']('https://macavaney.us/scibert-medmarco.tar.gz') avec le paramètre expected_md5="854966d0b61543ffffa44cea627ab63b".

Synthétisez toutes les mesures d'évaluation dans un même tableau (bm25, knrm et Vanilla Bert / avec/sans entraînement).

In [ ]:
## VBert sans pré-traitement

In [ ]:
## Chargement du checkpoint

In [ ]:
## Comparaison des performances

# MonoT5

Pour expérimenter le modèle T5, on utilisera une librairie différente implémentée par pyTerrier. 

In [ ]:
from pyterrier_t5 import MonoT5ReRanker
monoT5 = MonoT5ReRanker(text_field='abstract')

br = pt.BatchRetrieve(index) % 30
pipeline = (br >> pt.text.get_text(dataset, 'abstract') >> monoT5)
pt.Experiment(
    [br, pipeline],
    dataset.get_topics('description'),
    dataset.get_qrels(),
    names=['DPH', 'DPH >> T5'],
    eval_metrics=["map", "recip_rank", "ndcg", "ndcg_cut.10", "mrt"]
)